In [1]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Demo to Show Anywidget mode

In [2]:
import bigframes.pandas as bpd

/usr/local/google/home/shuowei/src/python-bigquery-dataframes/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.15) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Set the display option to use anywidget

In [3]:
bpd.options.bigquery.ordering_mode = "partial"
bpd.options.display.repr_mode = "anywidget"

Load Sample Data

In [4]:
df = bpd.read_gbq("bigquery-public-data.usa_names.usa_1910_2013")
print(df)

Computation deferred. Computation will process 171.4 MB


Display Series in anywidget mode

In [5]:
test_series = df["year"]
print(test_series)

Computation deferred. Computation will process 44.4 MB


Display with Pagination

In [6]:
df

TableWidget(page_size=10, row_count=5552452, table_html='<table border="1" class="dataframe table table-stripe…

Computation deferred. Computation will process 171.4 MB

Programmatic Navigation Demo

In [7]:
from bigframes.display.anywidget import TableWidget
import math
  
# Create widget programmatically  
widget = TableWidget(df)
print(f"Total pages: {math.ceil(widget.row_count / widget.page_size)}")
  
# Display the widget
widget

Total pages: 555246


TableWidget(page_size=10, row_count=5552452, table_html='<table border="1" class="dataframe table table-stripe…

Test Navigation Programmatically

In [8]:
# Simulate button clicks programmatically
print("Current page:", widget.page)

# Go to next page
widget.page = 1
print("After next:", widget.page)

# Go to previous page
widget.page = 0
print("After prev:", widget.page)

Current page: 0
After next: 1
After prev: 0


Edge Case Demonstration

In [9]:
# Test with very small dataset
small_df = df.sort_values(["name", "year", "state"]).head(5)
small_widget = TableWidget(small_df)
print(f"Small dataset pages: {math.ceil(small_widget.row_count / small_widget.page_size)}")
small_widget

Small dataset pages: 1


TableWidget(page_size=10, row_count=5, table_html='<table border="1" class="dataframe table table-striped tabl…

### Displaying Generative AI results containing JSON
The `AI.GENERATE` function in BigQuery returns results in a JSON column. While BigQuery's JSON type is not natively supported by the underlying Arrow `to_pandas_batches()` method used in anywidget mode ([Apache Arrow issue #45262](https://github.com/apache/arrow/issues/45262)), BigQuery Dataframes automatically converts JSON columns to strings for display. This allows you to view the results of generative AI functions seamlessly.

In [10]:
bpd._read_gbq_colab("""
  SELECT
    AI.GENERATE(
      prompt=>(\"Extract the values.\", OBJ.GET_ACCESS_URL(OBJ.FETCH_METADATA(OBJ.MAKE_REF(gcs_path, \"us.conn\")), \"r\")),
      connection_id=>\"bigframes-dev.us.bigframes-default-connection\",
      output_schema=>\"publication_date string, class_international string, application_number string, filing_date string\") AS result,
    *
  FROM `bigquery-public-data.labeled_patents.extracted_data`
  LIMIT 5;
""")

/usr/local/google/home/shuowei/src/github.com/googleapis/python-bigquery-dataframes/bigframes/dtypes.py:969: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)
/usr/local/google/home/shuowei/src/github.com/googleapis/python-bigquery-dataframes/bigframes/dataframe.py:867: UserWarning: Converting JSON columns to strings for display. This is temporary and will be removed when the frontend supports JSON types.
  warnings.warn(


TableWidget(page_size=10, row_count=5, table_html='<table border="1" class="dataframe table table-striped tabl…

Computation deferred. Computation will process 0 Bytes